In [1]:
import ee
import pandas as pd
import numpy as np
import folium
import time
import logging
import requests
import zipfile
import os
import math

ee.Initialize()

In [14]:
#Poligono referente a região do banhado
coordenadas = "-45.92492886742651,-23.199948257720962,-45.88098355492651,-23.160970895077618"
x1,y1,x2,y2 = coordenadas.split(",")
geometria = geometry = ee.Geometry.Polygon(
        [[[float(x1),float(y2)],
          [float(x2),float(y2)],
          [float(x2),float(y1)],
          [float(x1),float(y1)],
          [float(x1),float(y2)]]])
datasL5 = "2000-01-01,2013-01-01"
ini,fi = datasL5.split(",")
datasL8 = "2013-01-01,2021-04-14"
inicio,fim = datasL8.split(",")
#colecaoL5 = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR").filterBounds(geometria).filterDate(ini,fi).filterMetadata('CLOUD_COVER','less_than', 10)
colecaoL8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA").filterBounds(geometria).filterDate(inicio,fim).filterMetadata('CLOUD_COVER','less_than', 10)
#colecaoS2 = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(geometria).filterDate(inicio,fim).filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than', 5)
#print("LANDSAT 5: "+str(colecaoL5.size().getInfo()))
print("LANDSAT 8: "+str(colecaoL8.size().getInfo()))
#print("SENTINEL 2: "+str(colecaoS2.size().getInfo()))

LANDSAT 8: 77


In [15]:
def organiza(col,start,end):
  years = ee.List.sequence(start,end).getInfo()
  lista = ee.List([])
  for year in years:
    priano = col.filterDate(str(year)+'-01-01',str(year)+'-12-31').sort('CLOUD_COVER').first()
    if type(priano) == ee.image.Image:
      lista = lista.add(priano)

  return lista

In [16]:
Lista_L8 = organiza(colecaoL8,2013,2020)
Cole_L8 = ee.ImageCollection(Lista_L8)

In [17]:
def ymdList(imgcol):
    def iter_func(image, newlist):
        date = ee.Number.parse(image.date().format("YYYYMMdd"))
        newlist = ee.List(newlist)
        return ee.List(newlist.add(date).sort())
    ymd = imgcol.iterate(iter_func, ee.List([]))
    return list(ee.List(ymd).reduce(ee.Reducer.frequencyHistogram()).getInfo().keys())

In [18]:
ymdList(Cole_L8)

['20130512',
 '20140803',
 '20150714',
 '20160707',
 '20170905',
 '20180823',
 '20190817',
 '20200531']

In [19]:
import numpy as np
from osgeo import gdal
from osgeo import osr
import time


#----------------------------------------------------------------------
#Adaptação do exemplo disponibilizado em:
#https://mygeoblog.com/2017/10/06/from-gee-to-numpy-to-geotiff/
def save_gee_tiff(image,area,bandList,dummyValue,scaleValue,path_out):

    #image generation time
    #timedate = image.get('GENERATION_TIME').getInfo()
 
    #get the lat lon and add the bands/attributes/operations
    latlon = ee.Image.pixelLonLat().addBands( image.select(bandList) )
 
    #apply reducer to list
    latlon = latlon.reduceRegion(reducer=ee.Reducer.toList(), geometry=area, maxPixels=1e8, scale=scaleValue)
  
    #get data into different arrays
    lats = np.array((ee.Array(latlon.get('latitude')).getInfo()))
    lons = np.array((ee.Array(latlon.get('longitude')).getInfo()))
    arrayList = []
    for i in range(len(bandList)):
        data = np.array((ee.Array(latlon.get(bandList[i])).getInfo()),np.float32)
        arrayList.append(data)

    #get the unique coordinates
    uniqueLats = np.unique(lats)
    uniqueLons = np.unique(lons)
 
    #get number of columns and rows from coordinates
    ncols = len(uniqueLons)    
    nrows = len(uniqueLats)
    #number of bands in output image
    nband = len(bandList)
 
    # determine pixelsizes
    ys = uniqueLats[1] - uniqueLats[0] 
    xs = uniqueLons[1] - uniqueLons[0]
 
    # create an array with dimensions of image
    arr = np.zeros([nband, nrows, ncols], np.float32) #-9999  #<<<descomentao para usar como dummy!?
 
    """ # fill the array with values
    counter = 0
    for y in range(nrows):       #linhas
        for x in range(ncols):   #colunas
            if (lats[counter] == uniqueLats[y]) and (lons[counter] == uniqueLons[x]) and (counter < (len(lats)-1)):
                #counter+=1
                for b in range(nband):         #bandas
                    arr[b,len(uniqueLats)-1-y,x] = arrayList[b][counter] # we start from lower left corner
                counter+=1 """
 
    #alternativa----------------------------------- (o código acima funciona na landsat)
    refLat = np.max(uniqueLats)
    refLon = np.min(uniqueLons)
    nItens = len(arrayList[0])
    #counter = 0
    for i in range(nItens):
        posLin = np.int64( np.round( (refLat - lats[i])/ys ) )
        #posCol = np.int64( np.round( (refLon - lons[i])/xs ) )
        posCol = np.int64( np.round( (lons[i] - refLon)/xs ) )
        for b in range(nband):
            arr[b,posLin,posCol] = arrayList[b][i]

    #SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    transform = (np.min(uniqueLons),xs,0,np.max(uniqueLats),0,-ys)
 
    #set the coordinate system
    target = osr.SpatialReference()
    target.ImportFromEPSG(4326)
 
    #set driver
    driver = gdal.GetDriverByName('GTiff')
 
    #timestring = time.strftime("%Y%m%d_%H%M%S")
    outDs = driver.Create(path_out, ncols,nrows,nband,gdal.GDT_Float32)

    #projection settings
    outDs.SetGeoTransform(transform)
    outDs.SetProjection(target.ExportToWkt())

    #building output image
    ind = 1
    for b in range(nband):
        bandArr = np.copy(arr[b,:,:])
        outBand = outDs.GetRasterBand(ind)
        outBand.WriteArray(bandArr)
        outBand.FlushCache()
        outBand.SetNoDataValue(dummyValue)
        ind += 1

    outDs = None
    del outDs, outBand

    return 'ok...'

In [20]:
ee.Image(Lista_L8.get(3)).bandNames().getInfo()

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'BQA']

In [34]:
hora = []
for i in range(Cole_L8.size().getInfo()):
    qualquer = ee.Image(Lista_L8.get(i)).getInfo()
    hora.append(qualquer['properties']['SCENE_CENTER_TIME'])

hora

['13:06:17.5161250Z',
 '13:04:21.1890800Z',
 '12:57:43.2669820Z',
 '13:04:15.5482330Z',
 '12:58:17.3685910Z',
 '12:57:40.5732640Z',
 '13:04:30.2782240Z',
 '13:03:51.0658569Z']

In [36]:
mediaL8 = colecaoL8.mean()
bandas = mediaL8.bandNames().getInfo()
bandas

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'BQA']

In [21]:
bandlist = ['B2','B3','B4','B8','B10','B11']
path = r'C:\Users\vinim\OneDrive\Área de Trabalho\VINICIUS\1_Sem_2021\PMA\Imagens\Landsat\\'
for i in range(Cole_L8.size().getInfo()):
  imageInfo = ee.Image(Lista_L8.get(i)).getInfo()
  image = ee.Image(-99999).blend(ee.Image(Lista_L8.get(i)))
  path_out_img = path + imageInfo['id'].split('/')[-1] +'.tif'
  print('Processando...',imageInfo['id'])
  save_gee_tiff(image,geometria,bandlist,-99999,30,path_out_img)

Processando... LANDSAT/LC08/C01/T1_TOA/LC08_219076_20130512
Processando... LANDSAT/LC08/C01/T1_TOA/LC08_219076_20140803
Processando... LANDSAT/LC08/C01/T1_TOA/LC08_218076_20150714
Processando... LANDSAT/LC08/C01/T1_TOA/LC08_219076_20160707
Processando... LANDSAT/LC08/C01/T1_TOA/LC08_218076_20170905
Processando... LANDSAT/LC08/C01/T1_TOA/LC08_218076_20180823
Processando... LANDSAT/LC08/C01/T1_TOA/LC08_219076_20190817
Processando... LANDSAT/LC08/C01/T1_TOA/LC08_219076_20200531


In [37]:
imageInfo = mediaL8.getInfo()
path_out_img = path + 'media_2013_2020' +'.tif'
print('Processando...',path_out_img)
save_gee_tiff(mediaL8,geometria,bandas,-99999,30,path_out_img)

Processando... C:\Users\vinim\OneDrive\Área de Trabalho\VINICIUS\1_Sem_2021\PMA\Imagens\Landsat\\media_2013_2020.tif


'ok...'